In [35]:
import skimage.io
import numpy as np
import matplotlib as plt

In [36]:
# Project Image Filtering and Hybrid Images Stencil Code
# Based on previous and current work
# by James Hays for CSCI 1430 @ Brown and
# CS 4495/6476 @ Georgia Tech
import numpy as np
from numpy import pi, exp, sqrt
from skimage import io, img_as_ubyte, img_as_float32
from skimage.transform import rescale
import cv2
import matplotlib.pyplot as plt
import math 

def my_imfilter(img: np.ndarray, filter: np.ndarray):   
    # Your code here #    
    img_rows=img.shape[0]
    img_col=img.shape[1]
    filtered_image = np.zeros_like(img)
    #rotate filter 90 degrees horizontally and vertically 
    filter=np.rot90(filter,2)
    #To gurantee odd filters
    if(((filter.shape[0])%2)==0 or ((filter.shape[1])%2)==0):
        #print('Error,filter dimensions should be odd !')
        raise Exception('Error,filter dimensions should be odd !,output will be undefined ')
    filter_rows=filter.shape[0]
    filter_cols=filter.shape[1]
    padded_img=np.zeros((img_rows+filter_rows-1,img_col+filter_cols-1))
    #padded_img rows R and columns C 
    padded_img_R=padded_img.shape[0]
    padded_img_C=padded_img.shape[1]
    padded_img[((filter_rows)//2):padded_img_R-((filter_rows)//2),((filter_cols)//2):padded_img_C-((filter_cols)//2)]=img
    for i in range(((filter_rows)//2),padded_img_R-((filter_rows)//2)):
        for j in range(((filter_cols)//2),padded_img_C-((filter_cols)//2)):
            template=padded_img[i-((filter_rows)//2):(i+((filter_rows)//2)+1),j-((filter_cols)//2):(j+((filter_cols)//2)+1)]
            result=template*filter
            #element-wise multiplication filter && image
            filtered_image[i-((filter_rows)//2),j-((filter_cols)//2)]=result.sum()

    print('filtered_image',filtered_image.shape)
    #cv2_imshow(filtered_image)

    return filtered_image   

In [37]:
import cv2
import math
import numpy as np
def gen_hybrid_image(image1: np.ndarray, image2: np.ndarray, cutoff_frequency: float):
    
    assert image1.shape == image2.shape
    """
    
    Inputs:
    - image1 -> The image from which to take the low frequencies.
    - image2 -> The image from which to take the high frequencies.
    - cutoff_frequency -> The standard deviation, in pixels, of the Gaussian
                         blur that will remove high frequencies.  

    Task:
    - Use my_imfilter to create 'low_frequencies' and 'high_frequencies'.
    - Combine them to create 'hybrid_image'.
    """
    image1 = Image.open(image1)
    image1 = np.asarray(image1)
    
    image2 = Image.open(image2)
    image2 = np.asarray(image2)
    
    filter_size = 2 * int(4 * cutoff_frequency + 0.5) + 1
    gaussian_filter = np.zeros((filter_size, filter_size), np.float32)
    
    m=filter_size//2
    n=filter_size//2
    
    for x in range(-m,m+1):
        for y in range(-n,n+1):         
            c1 = math.sqrt(2*(math.pi))
            constant=c1*cutoff_frequency
            F = np.exp(-(x**2 + y**2)/(2*(math.sqrt(cutoff_frequency))))
            gaussian_filter[x+m, y+n] = (1/constant)*F           
            gaussian_filter[x+m,y+n] = (1/(constant))*(G)
               
                       
    filtered_image1 = my_imfilter(image1,gaussian_filter)
    filtered_image2 =  image2 - my_imfilter(image2, gaussian_filter)

    hybrid_image = filtered_image1 + filtered_image2
    hybrid_image = np.clip(hybrid_image, 0.0, 1.0)

    low_frequencies = filtered_image1
    high_frequencies = filtered_image2 
    return low_frequencies, high_frequencies, hybrid_image                                  
  


In [38]:
####        CODE HERE        ####

def vis_hybrid_image(hybrid_image: np.ndarray):
    
    """
    Visualize a hybrid image by progressively downsampling the image and
    concatenating all of the images together.
    """
    scales = 5
    scale_factor = 0.5
    padding = 5
    original_height = hybrid_image.shape[0]
    num_colors = 1 if hybrid_image.ndim == 2 else 3

    output = np.copy(hybrid_image)
    cur_image = np.copy(hybrid_image)
    for scale in range(2, scales+1):
      # add padding
      output = np.hstack((output, np.ones((original_height, padding, num_colors),dtype=np.float32)))
      # downsample image
      cur_image = rescale(cur_image, scale_factor, mode='reflect')
      # pad the top to append to the output
      pad = np.ones((original_height-cur_image.shape[0], cur_image.shape[1],num_colors), dtype=np.float32)
      tmp = np.vstack((pad, cur_image))
      output = np.hstack((output, tmp))
    return output

def load_image(path):
    return img_as_float32(io.imread(path))

def save_image(path, im):
    return io.imsave(path, img_as_ubyte(im.copy()))